In [ ]:
#Connecting to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Labraries
!pip install -U albumentations
import albumentations as A
import cv2
import numpy as np
import os
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.3/165.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 24.7 MB/s eta 0:00:00
  Attempting uninstall: imageio
    Found existing installation: imageio 2.31.6
    Uninstalling imageio-2.31.6:
      Successfully uninstalled imageio-2.31.6
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.19.3
    Uninstalling scikit-image-0.19.3:
      Successfully uninstalled scikit-image-0.19.3
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.3.1
    Uninstalling albumentations-1.3.1:
      Successfully uninstal

In [ ]:
#Path to images and results
input_folder = '/content/drive/My Drive/Cancer/SCC'
output_folder = '/content/drive/My Drive/Cancer/AugImages/AugSCC'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
#Checking the path
print(f"Input folder: {input_folder}")
print(f"Output folder: {output_folder}")
assert os.path.exists(input_folder), "Input folder does not exist"
assert os.path.exists(output_folder), "Output folder does not exist"

Input folder: /content/drive/My Drive/Cancer/SCC
Output folder: /content/drive/My Drive/Cancer/AugImages/AugSCC


In [ ]:
#Images loading
def load_image(filepath):
    image = cv2.imread(filepath)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
#Images saving
def save_image(filepath, image):
    cv2.imwrite(filepath, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

In [ ]:
#Parameter tuning
transform = A.Compose([
    A.Rotate(limit=45, p=1.0),  #Rotation within the range of -45 to 45 degrees
    A.RandomScale(scale_limit=(0.2, 1.0), p=1.0),  #Resizing from 20% to 100%
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.0, rotate_limit=0, p=1.0)  #Shifting within the range of -3 to 3 pixels (0.05 of the image size)
])


In [ ]:
#Loading all images from a folder
image_paths = [os.path.join(input_folder, fname) for fname in os.listdir(input_folder) if fname.endswith('.jpg')]


In [ ]:
#Number of augmented images
num_augmented_images = 250
num_augmentations_per_image = num_augmented_images // len(image_paths)

In [ ]:
#Applying augmentation and saving the new images
for img_path in tqdm(image_paths):
    image = load_image(img_path)
    base_name = os.path.basename(img_path).split('.')[0]

    for i in range(num_augmentations_per_image):
        transformed = transform(image=image)
        transformed_image = transformed['image']
        new_image_name = f"{base_name}_aug_{i}.jpg"
        save_image(os.path.join(output_folder, new_image_name), transformed_image)

100%|██████████| 74/74 [01:47<00:00,  1.45s/it]


In [ ]:
#Processing the remainder
remainder = num_augmented_images % len(image_paths)
if remainder > 0:
    for i in range(remainder):
        img_path = image_paths[i % len(image_paths)]
        image = load_image(img_path)
        base_name = os.path.basename(img_path).split('.')[0]
        transformed = transform(image=image)
        transformed_image = transformed['image']
        new_image_name = f"{base_name}_aug_extra_{i}.jpg"
        save_image(os.path.join(output_folder, new_image_name), transformed_image)